# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5a18098c40>
├── 'a' --> tensor([[ 0.0130,  1.3219, -0.6813],
│                   [ 0.3578, -0.1146, -0.7528]])
└── 'x' --> <FastTreeValue 0x7f5a18098580>
    └── 'c' --> tensor([[ 0.0226,  0.9629, -0.7110, -0.5091],
                        [-0.1055, -0.6411, -2.8163, -0.1155],
                        [ 0.6048, -0.2691,  0.1120, -1.1461]])

In [4]:
t.a

tensor([[ 0.0130,  1.3219, -0.6813],
        [ 0.3578, -0.1146, -0.7528]])

In [5]:
%timeit t.a

63.2 ns ± 0.0614 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5a18098c40>
├── 'a' --> tensor([[ 0.5922,  0.2726,  0.0290],
│                   [ 0.6049, -2.3230, -1.6912]])
└── 'x' --> <FastTreeValue 0x7f5a18098580>
    └── 'c' --> tensor([[ 0.0226,  0.9629, -0.7110, -0.5091],
                        [-0.1055, -0.6411, -2.8163, -0.1155],
                        [ 0.6048, -0.2691,  0.1120, -1.1461]])

In [7]:
%timeit t.a = new_value

73.8 ns ± 0.0463 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0130,  1.3219, -0.6813],
               [ 0.3578, -0.1146, -0.7528]]),
    x: Batch(
           c: tensor([[ 0.0226,  0.9629, -0.7110, -0.5091],
                      [-0.1055, -0.6411, -2.8163, -0.1155],
                      [ 0.6048, -0.2691,  0.1120, -1.1461]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0130,  1.3219, -0.6813],
        [ 0.3578, -0.1146, -0.7528]])

In [11]:
%timeit b.a

59.8 ns ± 0.047 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.9529, -0.1280,  1.2021],
               [-1.6331, -0.6899,  0.5073]]),
    x: Batch(
           c: tensor([[ 0.0226,  0.9629, -0.7110, -0.5091],
                      [-0.1055, -0.6411, -2.8163, -0.1155],
                      [ 0.6048, -0.2691,  0.1120, -1.1461]]),
       ),
)

In [13]:
%timeit b.a = new_value

493 ns ± 0.0972 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

876 ns ± 0.192 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 20.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

146 µs ± 581 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 199 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f5a18056040>
├── 'a' --> tensor([[[ 0.0130,  1.3219, -0.6813],
│                    [ 0.3578, -0.1146, -0.7528]],
│           
│                   [[ 0.0130,  1.3219, -0.6813],
│                    [ 0.3578, -0.1146, -0.7528]],
│           
│                   [[ 0.0130,  1.3219, -0.6813],
│                    [ 0.3578, -0.1146, -0.7528]],
│           
│                   [[ 0.0130,  1.3219, -0.6813],
│                    [ 0.3578, -0.1146, -0.7528]],
│           
│                   [[ 0.0130,  1.3219, -0.6813],
│                    [ 0.3578, -0.1146, -0.7528]],
│           
│                   [[ 0.0130,  1.3219, -0.6813],
│                    [ 0.3578, -0.1146, -0.7528]],
│           
│                   [[ 0.0130,  1.3219, -0.6813],
│                    [ 0.3578, -0.1146, -0.7528]],
│           
│                   [[ 0.0130,  1.3219, -0.6813],
│                    [ 0.3578, -0.1146, -0.7528]]])
└── 'x' --> <FastTreeValue 0x7f596de105e0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 74.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f596dd1d970>
├── 'a' --> tensor([[ 0.0130,  1.3219, -0.6813],
│                   [ 0.3578, -0.1146, -0.7528],
│                   [ 0.0130,  1.3219, -0.6813],
│                   [ 0.3578, -0.1146, -0.7528],
│                   [ 0.0130,  1.3219, -0.6813],
│                   [ 0.3578, -0.1146, -0.7528],
│                   [ 0.0130,  1.3219, -0.6813],
│                   [ 0.3578, -0.1146, -0.7528],
│                   [ 0.0130,  1.3219, -0.6813],
│                   [ 0.3578, -0.1146, -0.7528],
│                   [ 0.0130,  1.3219, -0.6813],
│                   [ 0.3578, -0.1146, -0.7528],
│                   [ 0.0130,  1.3219, -0.6813],
│                   [ 0.3578, -0.1146, -0.7528],
│                   [ 0.0130,  1.3219, -0.6813],
│                   [ 0.3578, -0.1146, -0.7528]])
└── 'x' --> <FastTreeValue 0x7f597494b580>
    └── 'c' --> tensor([[ 0.0226,  0.9629, -0.7110, -0.5091],
                        [-0.1055, -0.6411, -2.8163, -0.1155],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 151 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.1 µs ± 54.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.0130,  1.3219, -0.6813],
                [ 0.3578, -0.1146, -0.7528]],
       
               [[ 0.0130,  1.3219, -0.6813],
                [ 0.3578, -0.1146, -0.7528]],
       
               [[ 0.0130,  1.3219, -0.6813],
                [ 0.3578, -0.1146, -0.7528]],
       
               [[ 0.0130,  1.3219, -0.6813],
                [ 0.3578, -0.1146, -0.7528]],
       
               [[ 0.0130,  1.3219, -0.6813],
                [ 0.3578, -0.1146, -0.7528]],
       
               [[ 0.0130,  1.3219, -0.6813],
                [ 0.3578, -0.1146, -0.7528]],
       
               [[ 0.0130,  1.3219, -0.6813],
                [ 0.3578, -0.1146, -0.7528]],
       
               [[ 0.0130,  1.3219, -0.6813],
                [ 0.3578, -0.1146, -0.7528]]]),
    x: Batch(
           c: tensor([[[ 0.0226,  0.9629, -0.7110, -0.5091],
                       [-0.1055, -0.6411, -2.8163, -0.1155],
                       [ 0.6048, -0.2691,  0.1120, -1.1461]],
         

In [26]:
%timeit Batch.stack(batches)

77.4 µs ± 178 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.0130,  1.3219, -0.6813],
               [ 0.3578, -0.1146, -0.7528],
               [ 0.0130,  1.3219, -0.6813],
               [ 0.3578, -0.1146, -0.7528],
               [ 0.0130,  1.3219, -0.6813],
               [ 0.3578, -0.1146, -0.7528],
               [ 0.0130,  1.3219, -0.6813],
               [ 0.3578, -0.1146, -0.7528],
               [ 0.0130,  1.3219, -0.6813],
               [ 0.3578, -0.1146, -0.7528],
               [ 0.0130,  1.3219, -0.6813],
               [ 0.3578, -0.1146, -0.7528],
               [ 0.0130,  1.3219, -0.6813],
               [ 0.3578, -0.1146, -0.7528],
               [ 0.0130,  1.3219, -0.6813],
               [ 0.3578, -0.1146, -0.7528]]),
    x: Batch(
           c: tensor([[ 0.0226,  0.9629, -0.7110, -0.5091],
                      [-0.1055, -0.6411, -2.8163, -0.1155],
                      [ 0.6048, -0.2691,  0.1120, -1.1461],
                      [ 0.0226,  0.9629, -0.7110, -0.5091],
                      [-0.1055, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 332 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

321 µs ± 2.54 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
